In [1]:
from tensorflow.keras.layers import LSTM

class CustomLSTM(LSTM):
    def __init__(self, *args, **kwargs):
        kwargs.pop('time_major', None)  # Remove the 'time_major' argument if it exists
        super(CustomLSTM, self).__init__(*args, **kwargs)

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'CustomLSTM': CustomLSTM})

trained_model = load_model(r"C:\Users\pranali\Downloads\5g_classifier.h5", custom_objects={'LSTM': CustomLSTM})

trained_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_0 (CustomLSTM)             │ (None, 10, 120)        │        63,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (CustomLSTM)             │ (None, 10, 160)        │       179,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 160)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (CustomLSTM)             │ (None, 10, 120)        │       134,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_0 (Dense)                 │ (None, 56)             │        67,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │         2,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         2,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OUT_multivariate_classifier     │ (None, 3)              │           147 │
│ (Dense)                         │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 450,573 (1.72 MB)

 Trainable params: 450,571 (1.72 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
import numpy as np
import tensorflow as tf

X_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\Mature(1)\Mature-main\Datasets\Testing\all_data_classifier_test_x.npy")
y_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\Mature(1)\Mature-main\Datasets\Testing\all_data_classifier_test_y.npy")

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)


In [5]:
import tensorflow as tf

# Convert the model to a TensorFlow Lite model with Float16 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(trained_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enable Float16 quantization
converter.target_spec.supported_types = [tf.float16]

# Enable Select TF ops to support operations not natively supported by TFLite
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
                                       tf.lite.OpsSet.SELECT_TF_OPS]

# Disable lowering of tensor list operations
converter._experimental_lower_tensor_list_ops = False

# Enable resource variable supporta
converter.experimental_enable_resource_variables = True

# Convert the model
tflite_float16_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\pranali\AppData\Local\Temp\tmpxd2u2s1p\assets


INFO:tensorflow:Assets written to: C:\Users\pranali\AppData\Local\Temp\tmpxd2u2s1p\assets


Saved artifact at 'C:\Users\pranali\AppData\Local\Temp\tmpxd2u2s1p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10, 11), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1393870104208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871372560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393870109968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393870110160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871375248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871375440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871376976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871377936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871375056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393871379088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139387138

In [6]:
# Save the TFLite model
with open('5G_classifier_f16.tflite', 'wb') as f:
    f.write(tflite_float16_model)

In [7]:
import tensorflow as tf
import numpy as np

# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_path='5G_classifier_f16.tflite')
interpreter.allocate_tensors()

# Get details of input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [11]:
import time

# Initialize list to store predictions
predictions = []

# Measure inference time
start_time = time.time()
for i in range(X_test.shape[0]):
    # Prepare the input data (trim the extra dimension if necessary)
    input_data = X_test[i]  # Trim to match the expected shape (1, 10, 10)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Append prediction to the list
    predictions.append(output_data)

end_time = time.time()
total_inference_time = end_time - start_time
avg_inference_time = total_inference_time / len(X_test)

print(f"Total Inference Time for quantized : {total_inference_time:.6f} seconds")
print(f"Avg Inference Time for quantized : {avg_inference_time:.6f} seconds")


Total Inference Time for quantized : 33.632126 seconds
Avg Inference Time for quantized : 0.004472 seconds


In [13]:
import os
# Get the size of the saved quantized TFLite model
model_size = os.path.getsize('5g_classifier_f16.tflite')
print(f"Quantized TFLite model size: {model_size / 1024:.2f} KB")  # Convert to KB

Quantized TFLite model size: 908.07 KB


In [15]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_float16_model)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run inference using the interpreter
def run_inference(interpreter, X_test):
    predictions = []
    for i in range(len(X_test)):
        # Preprocess the input as per your model's input requirements
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)  # Adjust datatype if needed
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(output_data)
    return np.array(predictions)

# Assume X_test and y_test are your test data and labels
# Run inference on the quantized model
predictions = run_inference(interpreter, X_test)

# Convert predictions to the correct shape if needed (e.g., flattening)
predictions = np.squeeze(predictions)

# For classification, get the predicted class and the true class
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Print the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Print the classification report
target_names = ["Low (0)", "Medium (1)", "High (2)"]
class_report = classification_report(true_classes, predicted_classes, target_names=target_names)
print("\nClassification Report:")
print(class_report)

# Optional: If you need to compute accuracy again
accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
print("Accuracy:", accuracy)

# Run inference on the quantized model
predictions = run_inference(interpreter, X_test)

# Convert predictions to the correct shape if needed (e.g., flattening)
predictions = np.squeeze(predictions)

# Compute loss (assuming Mean Squared Error for regression, adjust for classification)
mse = np.mean(np.square(predictions - y_test))
print("Mean Squared Error (MSE):", mse)


Confusion Matrix:
[[ 268   81   24]
 [  60  812  414]
 [  26  131 5704]]

Classification Report:
              precision    recall  f1-score   support

     Low (0)       0.76      0.72      0.74       373
  Medium (1)       0.79      0.63      0.70      1286
    High (2)       0.93      0.97      0.95      5861

    accuracy                           0.90      7520
   macro avg       0.83      0.77      0.80      7520
weighted avg       0.90      0.90      0.90      7520

Accuracy: 0.902127659574468
Mean Squared Error (MSE): 0.04958692824361586
